# getting labels

In [1]:
import geopy 
import pandas as pd
import sys
import time
import numpy as np
import IPython.core.display as dp

dataroot = 'C:/Users/lezhi/Dropbox/thesis/data/'
# tgtroot = 'C:/Users/lezhi/Dropbox/thesis/img/'
# dataroot = './'
def strcoor(coor):
    return str(coor[0])+','+str(coor[1])

In [134]:
coordf = pd.read_csv(dataroot + 'allcoorsvalid.csv')[['lat','lng','dir','city']].astype(str)
coordf.head()

,lat,lng,dir,city
0,41.341298,2.14397662154,0,barcelona
1,41.341298,2.14397662154,1,barcelona
2,41.341298,2.14397662154,2,barcelona
3,41.341298,2.14397662154,3,barcelona
4,41.342798,2.12798137831,0,barcelona


In [129]:
# unique lat-lng
uniq_coordf = coordf.drop_duplicates(subset=['lat','lng']).reset_index()
del uniq_coordf['index']
uniq_coordf.head()

,lat,lng,city
0,41.341298,2.14397662154,barcelona
1,41.342798,2.12798137831,barcelona
2,41.342798,2.12998078372,barcelona
3,41.342798,2.13198018912,barcelona
4,41.342798,2.13997781073,barcelona


In [130]:
lat_lng = zip(uniq_coordf["lat"], uniq_coordf["lng"])
lat_lng[:5], len(lat_lng)

([('41.341298', '2.14397662154'),
  ('41.342798', '2.12798137831'),
  ('41.342798', '2.12998078372'),
  ('41.342798', '2.13198018912'),
  ('41.342798', '2.13997781073')],
 34327)

In [5]:
# delete unnecessary fields
def del_redundant(dic, *args):
    for i in range(len(args)):
        name = args[i]
        if (name in dic):
            del dic[name]

In [6]:
def get_details(latlng_tup, index):
    geo_para = strcoor(latlng_tup)
    try:
        location = geolocator.reverse(geo_para)
    except:
        problem_list.append(index)
        # start_point = index
        print "Problem position:{}".format(index)
        raise
        return {"error": True}
    detail = location.raw["address"]
    del_redundant(detail, 'country_code', 'country', 'state', 'city')
    return detail

In [43]:
# this block should not be changed
end_point = len(lat_lng)
geolocator = geopy.geocoders.Nominatim()

#detail_df = pd.DataFrame()
#detail_df = pd.read_csv(dataroot + "results_all.csv", encoding='cp1252')
detail_df = pd.read_csv(dataroot + "results_all_original.csv", encoding='utf-8', index_col=0)
problem_list = []

In [29]:
start_point = 23000
start_time = time.time()
now_time = time.time()

detail_list = []
for i in range(start_point, end_point):      
    detail_list.append(get_details(lat_lng[i], i))
    
    if (i+1)%200 == 0 or i == (end_point - 1):         
#         try:
#             df = pd.DataFrame(detail_list)
#             detail_df = pd.concat([detail_df, df], axis=1)
#             detail_df.to_csv(dataroot + "results_all.csv")
#         except:
#             problem_list.append(i)
#             start_point = i
#             print "Has a problem in saving csv, position:{}".format(i)
#             raise
        
        df = pd.DataFrame.from_records(detail_list)
        detail_df = detail_df.append(df, ignore_index=True)
        detail_df.to_csv(dataroot + "results_all.csv", encoding='utf-8')
        
        dp.clear_output(wait=True)
        print "Ind: {}; Time for recent batch: {}".format(i, time.time()-now_time)
        sys.stdout.flush()
        
        detail_list = []
        now_time = time.time()
        
    time.sleep(0.7)
# 
# print "this turn takes:{}".format(time.time()-now_time)
# print "finish"
#geo_para, address_raw_dict
#detail_df

# making up error requests

In [8]:
detail_df = pd.read_csv(dataroot + "labels_all.csv", encoding='utf-8', index_col=0)

In [51]:
#details_df = pd.read_csv(dataroot + "labels_all.csv", encoding='utf-8', index_col=0)
error_ind = detail_df[detail_df['error'] == True].index
error_ind

Int64Index([22561, 22947, 22949, 22950, 22951, 22952, 22954, 22956, 22964,
            22975, 22976, 22977, 22978, 22979, 22981, 22982],
           dtype='int64')

In [52]:
makeup_list = []

for i in error_ind:
    makeup_list.append(get_details(lat_lng[i], i))
makeup_df = pd.DataFrame.from_records(makeup_list)
makeup_df['index'] = error_ind

In [69]:
makeup_df2 = makeup_df.set_index('index')
makeup_df2

,address29,city_district,county,footway,fuel,garden,hairdresser,house_number,marina,medical_centre,postcode,residential,road,suburb,town
index,,,,,,,,,,,,,,,
22561,NaN,14e,Paris,NaN,NaN,NaN,NaN,2,NaN,NaN,75014,NaN,Rue Lebouis,Plaisance,NaN
22947,NaN,12e,Paris,NaN,NaN,Jardin Mary Cassatt,NaN,NaN,NaN,NaN,75012,12e Arrondissement (urbain),Boulevard de Picpus,Picpus,NaN
22949,NaN,12e,Paris,NaN,NaN,NaN,NaN,73,NaN,NaN,75012,12e Arrondissement (urbain),Avenue du Docteur Arnold Netter,Bel-Air,NaN
22950,NaN,12e,Paris,NaN,NaN,NaN,NaN,30,NaN,NaN,75012,12e Arrondissement (urbain),Rue de la Voûte,Bel-Air,NaN
22951,NaN,12e,Paris,NaN,NaN,NaN,NaN,35,NaN,NaN,75012,12e Arrondissement (urbain),Rue Montéra,Bel-Air,NaN
22952,NaN,12e,Paris,NaN,NaN,NaN,NaN,9,NaN,NaN,75012,12e Arrondissement (urbain),Rue Fernand Foureau,Bel-Air,NaN
22954,NaN,NaN,Nogent-sur-Marne,NaN,NaN,NaN,Franck Provost,NaN,NaN,NaN,94160,NaN,Avenue Gallieni,NaN,Saint-Mandé
22956,NaN,NaN,Nogent-sur-Marne,NaN,NaN,NaN,NaN,97,NaN,NaN,94160,NaN,Avenue de Paris,NaN,Saint-Mandé
22964,NaN,15e,Paris,NaN,NaN,NaN,NaN,NaN,Port de Javel-Haut,NaN,75015,NaN,Port de Javel Haut,Javel,NaN


In [72]:
for i in error_ind:
    detail_df.loc[i] = makeup_df2.loc[i]

In [74]:
detail_df.to_csv(dataroot + 'labels.csv', encoding='utf-8')

## making up for japan!!! (see getting_label.ipynb)

# extracting useful label info

In [9]:
detail_df = pd.read_csv(dataroot + "labels.csv", encoding='utf-8', index_col=0)

C:\Anaconda\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (1,2,3,5,6,7,8,10,11,13,16,17,18,19,20,21,22,23,25,27,29,30,31,32,33,34,35,36,37,38,39,40,43,44,46,47,49,50,51,52,53,54,55,56,57,58,60,62,64,66,67,69,71,72,73,74,75,76,77,78,80,84,85,86,89,92,93,94,95,96,97,99,100,101,102,103,104,105,108,110,111,112,116,120,121,123,124,125,126,127,129,131,132,134,135,136,137,138,139,141,142,143,146,147,148,150,151,152,153,154,155,156,157,158,160,162,164,165,166) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [10]:
df_use = detail_df[['neighbourhood', 'suburb', 'town', 'postcode', 'road']]
df_use

,neighbourhood,suburb,town,postcode,road
0,NaN,Zona Franca,NaN,08040,NaN
1,NaN,Zona Franca,NaN,08040,Ronda Litoral
2,NaN,Zona Franca,NaN,08040,Ronda Litoral
3,NaN,Zona Franca,NaN,08040,Avinguda del Parc Logístic
4,NaN,Zona Franca,NaN,08040,Carrer Número 2
5,NaN,Zona Franca,NaN,08040,Carrer Número 2
6,NaN,Zona Franca,NaN,08040,Ronda Litoral
7,NaN,Zona Franca,NaN,08040,Ronda Litoral
8,NaN,Zona Franca,NaN,08040,Carrer Número 2
9,NaN,Zona Franca,NaN,08040,Carrer Número 2


In [38]:
# 184 points don't have any "neighbourhood" information
sum(df_use[['neighbourhood', 'suburb', 'town', 'postcode']].isnull().all(axis=1))

1540

In [44]:
null_number = df_use[['neighbourhood', 'suburb', 'town']].isnull().apply(np.sum, axis=1)

In [49]:
multilabel_df = df_use[['neighbourhood', 'suburb', 'town']][null_number <= 1]
multilabel_df.to_csv(dataroot + 'multilabel.csv',  encoding='utf-8')

In [40]:
# number of nulls in a row
null_number = pd.DataFrame(null_number, columns=["num_null"]).reset_index()
null_number.groupby('num_null').agg(lambda x: len(x))

,index
num_null,
0,1
1,10105
2,20013
3,4208


In [43]:
df_use[['neighbourhood', 'suburb', 'town']].isnull().apply(np.sum, axis=0)

neighbourhood    18449
suburb           12673
town             31633
dtype: int64

In [56]:
len(coordf), len(df_use), full_df.dtypes

(34327, 34327, index             int64
 lat              object
 lng              object
 city             object
 neighbourhood    object
 suburb           object
 town             object
 postcode         object
 road             object
 dtype: object)

In [57]:
full_df = pd.merge(coordf.reset_index(), df_use.reset_index(), on='index')
del full_df['index'] 

In [61]:
full_df.to_csv(dataroot + 'coor_with_multilabel.csv', encoding='utf-8')#[full_df['city']=='tokyo']

In [91]:
def get_label(row):
    if row['city'] == 'boston' or row['city'] == 'newyork' or row['city'] == 'chicago':
        if not pd.isnull(row['neighbourhood']):
            return row['neighbourhood']        
        elif not pd.isnull(row['suburb']):
            return row['suburb']
        elif not pd.isnull(row['town']):
            return row['town']
    else:
        if not pd.isnull(row['suburb']):
            return row['suburb']
        elif not pd.isnull(row['neighbourhood']):
            return row['neighbourhood']
        elif not pd.isnull(row['town']):
            return row['town']
    return 'nothing'

In [92]:
full_df['label'] = full_df.apply(get_label, axis=1)

## try out on Boston and Singapore first

In [118]:
bos_sin = full_df[full_df.apply(lambda x: x['city'] == 'boston' or x['city'] == 'singapore', axis=1)][['lat','lng','city','label']]

In [119]:
len(bos_sin['label'].unique())

103

In [113]:
labelnum = bos_sin['label'].reset_index().groupby('label').agg(lambda x: len(x)).reset_index().rename(columns={'index':'num'})

In [120]:
a = pd.merge(bos_sin, labelnum, on='label')

In [122]:
a.to_csv('test.csv')

In [124]:
a.head()

,lat,lng,city,label,num
0,42.298631,-71.0830938769,boston,Mount Bowdoin,48
1,42.298631,-71.0851235962,boston,Mount Bowdoin,48
2,42.298631,-71.0871533155,boston,Mount Bowdoin,48
3,42.298631,-71.0891830348,boston,Mount Bowdoin,48
4,42.300131,-71.0729452804,boston,Mount Bowdoin,48


In [139]:
b = pd.merge(coordf, a[['lat','lng','label']], on=['lat','lng'])
b['filename'] = b.apply(lambda x: x['city']+'/'+x['lat']+','+x['lng']+'_'+x['dir']+'.png', axis=1)

In [140]:
b.to_csv(dataroot + 'bos_sin_labels.csv')

In [141]:
b

,lat,lng,dir,city,label,filename
0,42.298631,-71.0830938769,0,boston,Mount Bowdoin,"boston/42.298631,-71.0830938769_0.png"
1,42.298631,-71.0830938769,1,boston,Mount Bowdoin,"boston/42.298631,-71.0830938769_1.png"
2,42.298631,-71.0830938769,2,boston,Mount Bowdoin,"boston/42.298631,-71.0830938769_2.png"
3,42.298631,-71.0830938769,3,boston,Mount Bowdoin,"boston/42.298631,-71.0830938769_3.png"
4,42.298631,-71.0851235962,0,boston,Mount Bowdoin,"boston/42.298631,-71.0851235962_0.png"
5,42.298631,-71.0851235962,1,boston,Mount Bowdoin,"boston/42.298631,-71.0851235962_1.png"
6,42.298631,-71.0851235962,2,boston,Mount Bowdoin,"boston/42.298631,-71.0851235962_2.png"
7,42.298631,-71.0851235962,3,boston,Mount Bowdoin,"boston/42.298631,-71.0851235962_3.png"
8,42.298631,-71.0871533155,0,boston,Mount Bowdoin,"boston/42.298631,-71.0871533155_0.png"
9,42.298631,-71.0871533155,1,boston,Mount Bowdoin,"boston/42.298631,-71.0871533155_1.png"


In [68]:
get_label(full_df.loc[23396])

u'Beaubourg'

In [12]:
sum(df_use['postcode'].isnull())

1540

In [15]:
pd.isnull(df_use.loc[34326,'suburb'])

True

In [ ]:
labels = df_use.apply()